# odinson
Retrieve original Marvel Comics editions for stories published in old brazilian comics

##### 2021.11.14

---

## Souces

[Guia dos Quadrinhos](http://www.guiadosquadrinhos.com/)
- [Heróis da TV](http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274)
- [Superaventuras Marvel](http://www.guiadosquadrinhos.com/edicao/superaventuras-marvel-n-1/sam0301/7927)
- [Almanaque Premiere Marvel](http://www.guiadosquadrinhos.com/edicao/almanaque-premiere-marvel-n-1/apm02001/18611)

---


## 1. Develop retrieving:
- edition
    - id="nome_titulo_lb"
- ~cover image? link?~
    - id="ampliar_capa"
- story (portuguese)
    - all: id="texto_pag_detalhe"
- main character
- original publication
- year
- original title

In [ ]:
urlroot = "http://www.guiadosquadrinhos.com/"
# urledition = "edicao/herois-da-tv-2-serie-n-1/htv0302/6274"
urledition = "edicao/superaventuras-marvel-n-1/sam0301/7927"

In [ ]:
urlroot + urledition

### 1.1 dev scrape main #1 page

In [ ]:
# dev scraping
import requests
from bs4 import BeautifulSoup

URL = urlroot + urledition
page = requests.get(URL)

thesoup = BeautifulSoup(page.content, "html.parser")

#### 1.1.1 Edition

In [ ]:
# Edition
def edition(asoup):
    ed = asoup.find(id="nome_titulo_lb")
    edl = ed.contents
    edl = [f"{i}".strip() for i in edl]
    edl = [i.split(">")[1] if ">" in i else i for i in edl]
    edl = [i.split("<")[0] if "<" in i else i for i in edl]
    edl = [i.replace("n°", "").strip() for i in edl if i != '']
    return edl

In [ ]:
ed = edition(thesoup)
ed

#### 1.1.2 Edition Contents

In [ ]:
def contents(asoup):
    results = asoup.find(id="texto_pag_detalhe")
    results = results.text
    return results

In [ ]:
editioncontents = contents(thesoup)
print(editioncontents)

#### 1.1.3 Stories

In [ ]:
# All stories (individual)
def stories(asoup):
    storiesdict = {}
    # page section
    results = asoup.find(id="texto_pag_detalhe")
    # individual stories
    stories = results.find_all("div", class_="historia")
#     storieslist = [i.text for i in stories]
    for i in range(len(stories)):
        storiesdict[i] = stories[i].text
    return storiesdict

In [ ]:
stories = stories(thesoup)
stories

#### 1.1.4 Story Details

In [ ]:
def storydetails(contents, stories, storynumber):
    # return details only if story exists
    if (storynumber == 0 or storynumber > len(stories)):
        storydetails = None
    # for any story before the last
    elif storynumber < len(stories):
        storydetails = contents.split(stories[storynumber-1])[1].split(stories[storynumber])[0]
    # for the last story
    elif storynumber == len(stories):
        storydetails = contents.split(stories[storynumber-1])[1]
    return storydetails

In [ ]:
thestorydetail = storydetails(editioncontents, stories, 4)
print(thestorydetail)

#### 1.1.5 Main Character

In [ ]:
def maincharacter(astorydetail):
    character = astorydetail.split(",")[0].split("Personagens:")[1].strip()
    return character

In [ ]:
thecharacter = maincharacter(thestorydetail)
thecharacter

#### 1.1.6 Original Title

In [ ]:
def originaltitle(astorydetail):
    title = astorydetail.split("Publicada originalmente em")[1].split("(")[0].strip()
    return title

In [ ]:
theoriginal = originaltitle(thestorydetail)
theoriginal

#### 1.1.7 Original Number

In [ ]:
def originalnumber(astorydetail):
    issue = astorydetail.split("Publicada originalmente em")[1].split(")")[1].replace("n°\xa0", "").split("/")[0].strip()
    return issue

In [ ]:
thenumber = originalnumber(thestorydetail)
thenumber

#### 1.1.8 Original Year

In [ ]:
def originalyear(astorydetail):
    year = astorydetail.split("Publicada originalmente em")[1].split(")")[1].replace("n°\xa0", "").split("/")[1].split()[0].strip()
    return year

In [ ]:
theyear = originalyear(thestorydetail)
theyear

## 2 retrieve list of links for all editions

In [ ]:
def navigator(asoup):
    outlist = []
    nav = asoup.find(id="d0")
    for link in nav.find_all("a"):
        outlist.append(f'{link["href"].replace("../", "")}')
    return outlist

In [ ]:
alleditions = navigator(thesoup)

In [ ]:
alleditions

## 3. Save CSV and print log

In [ ]:
dict1 = {1:("a", "b"), 2:("c", "d")}

In [ ]:
dict1[2][0]

In [ ]:
list1 = [10, 9, 8]
list1.insert(0, 1)
list1

## 4. Test

In [1]:
from odinson import *

In [2]:
# od = Odinson("http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274")
od = Odinson("http://www.guiadosquadrinhos.com/edicao/herois-da-tv-n-2/htv0302/6275")

 Retrieving page contents for
 http://www.guiadosquadrinhos.com/edicao/herois-da-tv-n-2/htv0302/6275
 ---------------------------------------------
  Heróis da TV #2
 ---------------------------------------------


In [ ]:
od.initialurl

In [ ]:
od.editionurl

In [ ]:
od.editionnumber

In [ ]:
od.editioncontents[:800]

In [ ]:
od.editioncontents

In [ ]:
od.storytitles

In [3]:
od.storydetails

{0: '\nPersonagens:  Capitão Marvel,  Rick Jones,  Hulk,  Mordecai Boggs,  Capitão América,  Vingadores,  Thor,  Homem de Ferro,  Gigante,  Yon-Rogg\nRoteiro:  Roy Thomas Desenho:  Gil Kane Arte-Final:  Dan Adkins Cores:  Rita de Carvalho Letrista:  José Belmiro Editor original:  Stan Lee\nPublicada originalmente em   Captain Marvel (1968) n°\xa020/1970 - Marvel Comics\r\n             \r\nTítulo original americano: “The hunter and the holocaust”.\r\n\r\n\r\n               ',
 1: '\nPersonagens:  Capitão Marvel,  Rick Jones,  Hulk\nRoteiro:  Roy Thomas Desenho:  Gil Kane Arte-Final:  Dan Adkins Cores:  Rita de Carvalho Letrista:  José Belmiro Editor original:  Stan Lee\nPublicada originalmente em   Captain Marvel (1968) n°\xa021/1970 - Marvel Comics\r\n             \r\nTítulo original americano: “Here comes the Hulk!”.\r\n\r\n\r\n               ',
 2: '\nPersonagens:  Homem de Ferro,  Fantasma (Sabotador),  Pepper Potts,  Happy Hogan\nRoteiro:  Stan Lee Desenho:  Don Heck Arte-Final:  D

In [4]:
# troubleshoot
i = 0
sumdict = od.storydetails[i].split('".')[1].split("\r\n")[0].strip()
sumdict

IndexError: list index out of range

In [ ]:
od.maincharacter

In [ ]:
od.originaltitle

In [ ]:
od.originalnumber

In [ ]:
od.editionnumber[0] 

In [ ]:
od.editiondf

In [ ]:
pd.concat

In [ ]:
import datetime

In [ ]:
datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d%H%M')

## 5. Navigate results

In [5]:
htv = pd.read_csv("Comics_HeroisDaTV_202111302051.csv")

In [6]:
htv

,Unnamed: 0,BRTITLE,BRNUM,STORYNUM,BRSTORY,MAINCHAR,ALLCHAR,ORIGTITLE,ORIGNUM,ORIGYEAR,WRITER,ARTIST,SUMMARY
0,0,Heróis da TV,1,1,O Devorador de Planetas,Thor,Thor Galactus Tana Nile Odin Sif Registrador 2...,Thor,160.0,1969.0,Stan Lee,Jack Kirby,Tana Nile pede ajuda ao deus do trovão para en...
1,1,Heróis da TV,1,2,Deve um deus prevalecer?,Thor,Thor Galactus Ego O Planeta Vivo Registrador 211,Thor,161.0,1969.0,Stan Lee,Jack Kirby,Confronto final entre Thor e as forças de Gala...
2,2,Heróis da TV,1,3,Duelo de ferro,Punho de Ferro,Punho de Ferro Homem de Ferro Rafael Scarfe Mi...,Iron Fist,1.0,1975.0,Chris Claremont,John Byrne,Nosso herói invade a propriedade de Tony Stark...
3,3,Heróis da TV,1,4,Amargo adeus,Homem de Ferro,Homem de Ferro Mecanóide,Iron Man,32.0,1970.0,Allyn Brodsky,George Tuska,O Ferroso é capturado por um andróide alieníge...
4,4,Heróis da TV,1,5,Meia-Noite traz a morte!,Shang-Chi - Mestre do Kung Fu,Shang-Chi - Mestre do Kung Fu Meia-Noite Fu Ma...,Special Marvel Edition,16.0,1974.0,Steve Englehart - ‘John Harkness’,Jim Starlin,Fu Manchu envia o filho adotivo para matar Sha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,414,Heróis da TV,100,4,Segredo diabólico,Homem de Ferro,Homem de Ferro Diablo,Iron Man,159.0,1982.0,Roger McKenzie,Paul Smith,NaN
415,415,Heróis da TV,100,5,X-Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,416,Heróis da TV,100,6,Animal ferido,Wolverine,Wolverine Chispinha Lady Letal Carniceiros Esp...,"Uncanny X-Men, The",205.0,1986.0,Chris Claremont Arte: Barry Windsor-Smith Cor...,NaN,Apesar do título original americano da revista...
417,417,Heróis da TV,100,7,A vinda dos vingadores,Vingadores,Vingadores Hulk Thor Homem de Ferro Homem-Form...,"Avengers, The",1.0,1963.0,Stan Lee,Jack Kirby,NaN


In [12]:
allmaincharacters = htv["MAINCHAR"].drop_duplicates().to_list()
allmaincharacters

['Thor',
 'Punho de Ferro',
 'Homem de Ferro',
 'Shang-Chi - Mestre do Kung Fu',
 'Surfista Prateado',
 'Capitão Marvel',
 'Campeões',
 'Shang-Chi - Mestre do Kung Fu\nRoteiro:  Gerry Conway',
 'Homem-Formiga',
 'Motoqueiro Fantasma\nRoteiro:  Bill Mantlo Desenho:  George Tuska Arte-Final:  Vince Colletta Editor original:  Marv Wolfman\nPublicada originalmente em   Ghost Rider (1973) n°\xa016/1976 - Marvel Comics\r\n             \r\nTítulo original: "Blood in the waters".',
 'Miss Marvel',
 'Capitão América',
 'Motoqueiro Fantasma',
 'Legião de Monstros',
 'Punho de Ferro\nRoteiro:  Tony Isabella Desenho:  Arvell Jones Arte-Final:  Vince Colletta Editor original:  Len Wein\nPublicada originalmente em   Marvel Premiere (1972) n°\xa021/1975 - Marvel Comics\r\n             \r\nTítulo original: "Daughters of the death-goddess".',
 'Punho de Ferro\nRoteiro:  Tony Isabella Desenho:  Arvell Jones Arte-Final:  Aubrey Bradford Editor original:  Len Wein\nPublicada originalmente em   Marvel Prem

In [14]:
from pandasgui import show

ModuleNotFoundError: No module named 'pandasgui'